## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,72.93,33,100,2.62,overcast clouds
1,1,Longyearbyen,SJ,78.2186,15.6401,8.46,88,87,1.86,overcast clouds
2,2,Broome,US,42.2506,-75.8330,35.55,70,8,3.00,clear sky
3,3,Senanga,ZM,-16.1167,23.2667,73.26,54,83,4.29,broken clouds
4,4,Hermanus,ZA,-34.4187,19.2345,55.90,76,11,5.01,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,11,Rikitea,PF,-23.1203,-134.9692,75.07,81,54,11.45,light rain
12,12,Meulaboh,ID,4.1363,96.1285,80.73,80,100,5.73,overcast clouds
26,26,Hilo,US,19.7297,-155.0900,80.33,74,1,8.05,clear sky
31,31,Hervey Bay,AU,-25.2986,152.8535,81.03,66,23,8.99,few clouds
32,32,Hobart,AU,-42.8794,147.3294,75.06,59,75,3.00,light rain
34,34,Avarua,CK,-21.2078,-159.7750,78.85,61,91,12.66,overcast clouds
37,37,Butaritari,KI,3.0707,172.7902,82.38,71,61,12.59,broken clouds
43,43,Santa Marta,CO,11.2408,-74.1990,80.65,78,20,4.61,few clouds
45,45,Bambous Virieux,MU,-20.3428,57.7575,75.22,94,75,3.44,broken clouds
48,48,Sur,OM,22.5667,59.5289,75.76,53,0,7.20,clear sky


In [14]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                185
City                   185
Country                185
Lat                    185
Lng                    185
Max Temp               185
Humidity               185
Cloudiness             185
Wind Speed             185
Current Description    185
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Rikitea,PF,75.07,light rain,-23.1203,-134.9692,
12,Meulaboh,ID,80.73,overcast clouds,4.1363,96.1285,
26,Hilo,US,80.33,clear sky,19.7297,-155.0900,
31,Hervey Bay,AU,81.03,few clouds,-25.2986,152.8535,
32,Hobart,AU,75.06,light rain,-42.8794,147.3294,
34,Avarua,CK,78.85,overcast clouds,-21.2078,-159.7750,
37,Butaritari,KI,82.38,broken clouds,3.0707,172.7902,
43,Santa Marta,CO,80.65,few clouds,11.2408,-74.1990,
45,Bambous Virieux,MU,75.22,broken clouds,-20.3428,57.7575,
48,Sur,OM,75.76,clear sky,22.5667,59.5289,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
hotel_df.count()

City                   185
Country                185
Max Temp               185
Current Description    185
Lat                    185
Lng                    185
Hotel Name             185
dtype: int64

In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()

In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
""" 


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))